In [1]:
import os
import pandas as pd
import pickle as pkl
from scipy import sparse as sp

data_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_user/"

search_path = os.path.join(data_dir, "train_searchs.csv")
sim_rec_path = os.path.join(data_dir, "train_sim_recs.csv") 
compl_rec_path = os.path.join(data_dir, "train_compl_recs.csv") 

sim_rec_df = pd.read_csv(sim_rec_path, index_col=0)
compl_rec_df = pd.read_csv(compl_rec_path, index_col=0)
search_df = pd.read_csv(search_path, index_col=0)

with open(os.path.join(data_dir, "user_to_uid.pkl"), "rb") as fin:
    user_to_uid = pkl.load(fin)
with open(os.path.join(data_dir, "ivm_to_pid.pkl"), "rb") as fin:
    ivm_to_pid = pkl.load(fin)
    
print("number of users, items = {:,}, {:,}".format(len(user_to_uid), len(ivm_to_pid)))

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


number of users, items = 893,619, 2,260,878


In [6]:
from scipy import sparse as sp

sim_rec_df["sim_pids"] = sim_rec_df["sim_pids"].apply(lambda x: eval(x))
sim_rec_df = sim_rec_df.explode("sim_pids")
sim_rec_df["sim_pids"] = sim_rec_df["sim_pids"].astype("int64")
sim_rec_df = sim_rec_df.drop_duplicates(["uid", "aid", "sim_pids"])

compl_rec_df["compl_pids"] = compl_rec_df["compl_pids"].apply(lambda x: eval(x))
compl_rec_df = compl_rec_df.explode("compl_pids")
compl_rec_df["compl_pids"] = compl_rec_df["compl_pids"].astype("int64")
compl_rec_df = compl_rec_df.drop_duplicates(["uid", "aid", "compl_pids"])

search_df["rel_pids"] = search_df["rel_pids"].apply(lambda x: eval(x))
search_df = search_df.explode("rel_pids")
search_df["rel_pids"] = search_df["rel_pids"].astype("int64")
search_df = search_df.drop_duplicates(["uid", "qid", "rel_pids"])

#uids, aids, sim_pids = np.array(sim_rec_df.uid), np.array(sim_rec_df.aid), np.array(sim_rec_df.sim_pids)
#assert len(uids) == len(aids) == len(sim_pids)
#assert type(uids[0]) == type(aids[0]) == type(sim_pids[0])
#print("uids, aids, sim_pids densities = {:.3f}, {:.3f}, {:.3f}".format(
#    len(uids) / len(np.unique(uids)), len(aids) / len(np.unique(aids)), len(sim_pids) / len(np.unique(sim_pids))
#))

#targets = np.ones(len(uids))
#ui_shape = (len(user_to_uid), len(ivm_to_pid))
#ai_shape = (len(ivm_to_pid), len(ivm_to_pid))

#ui_data = sp.coo_matrix((targets, (uids, sim_pids)), shape=ui_shape).tocsr()
#ai_data = sp.coo_matrix((targets, (aids, sim_pids)), shape=ai_shape).tocsr()

TypeError: eval() arg 1 must be a string, bytes or code object

In [10]:
SIM_RELATION = "is_similar_to"
COMPL_RELATION = "is_complementary_to"
REL_RELATION = "is_relevant_to"

sim_rec_df["relation"] = SIM_RELATION
compl_rec_df["relation"] = COMPL_RELATION
search_df["relation"] = REL_RELATION
sim_rec_df.rename({"sim_pids": "pid"}, axis=1, inplace=True)
compl_rec_df.rename({"compl_pids": "pid"}, axis=1, inplace=True)
search_df.rename({"rel_pids": "pid"}, axis=1, inplace=True)

merge_df = pd.concat([
    sim_rec_df[["uid", "pid", "relation"]],
    compl_rec_df[["uid", "pid", "relation"]],
    search_df[["uid", "pid", "relation"]]
])

REL_TO_ID = {
    SIM_RELATION: 0,
    COMPL_RELATION: 1,
    REL_RELATION: 2
}



In [23]:
import torch
import torch.nn as nn

from utils import  AverageMeter

class CFDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df
        
    def __getitem__(self, idx):
        return list(self.df.iloc[idx])
        
    def __len__(self):
        return len(self.df)
    
    def collate_fn(self, batch):
        uids, pids, rel_ids = [], [], []
        for uid, pid, rel in batch:
            uids.append(uid)
            pids.append(pid)
            rel_ids.append(REL_TO_ID[rel])
            
        return {
            "user_ids": torch.LongTensor(uids),
            "item_ids": torch.LongTensor(pids),
            "rel_ids": torch.LongTensor(rel_ids),
        }
    
class CF(torch.nn.Module):
    def __init__(self, ui_emb_size, rel_emb_size, num_user, num_item, dropout_rate=0.1):
        super().__init__()
        
        self.user_embs = nn.Embedding(num_user, ui_emb_size)
        self.item_embs = nn.Embedding(num_item, ui_emb_size)
        self.rel_embs = nn.Embedding(len(REL_TO_ID), rel_emb_size)
        
        self.dense = nn.Sequential(
                        nn.Linear(ui_emb_size+rel_emb_size, ui_emb_size),
                        nn.Dropout(p=dropout_rate),
                        nn.ReLU()
                    )
        
        self.loss = ContrastiveLoss()
        
    def forward(self, user_ids, item_ids, rel_ids):
        user_embs = self.query_embs(user_ids, rel_ids)
        item_embs = self.passage_embs(item_ids)
        
        return self.loss(user_embs, item_embs)
        
    def query_embs(self, user_ids, rel_ids):
        hidden_states = torch.cat([self.user_embs[user_ids], self.item_embs[item_ids]], dim=-1)
        hidden_states = self.dense(hidden_states)
        return hidden_states
    
    def passage_embs(self, item_ids):
        return self.item_embs(item_ids)

class ContrastiveLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss()
    
    def __call__(self, user_embs, item_embs):
        logits = user_embs @ item_embs
        targets = torch.arange(len(user_embs), dtype=torch.long).to(device=logits.device)
        
        return self.loss_fn(logits, targets)
    
def train(model, train_dataloader):
    loss_avg_meter = AverageMeter()
    global_step = 1
    print("batch: {}, step: {}, loss: {}")
    for epoch in range(128):
        for batch in train_dataloader:
            loss = model(**batch)
            loss_avg_meter.update(loss.item())
        
        if (global_step+1) % 4_000 == 0:
            print(f"{batch} {global_step} {loss_avg_meter.avg:.3f}")
            loss_avg_meter.reset()
            
        global_step += 1

train_dataset = CFDataset(merge_df)
train_dataloader = torch.utils.data.DataLoader(train_dataset, num_workers=4, shuffle=True, batch_size=128, 
                                              collate_fn=train_dataset.collate_fn)


{'uids': tensor([879029, 588698, 710444, 721113, 142077, 566914, 428127, 308658,  67313,
         13947, 328308, 412394, 273223, 797188, 831989, 509147, 497062, 354090,
        213729, 628916, 796984, 652821, 247175, 218926,   3382, 238957, 203408,
        190488,  40832, 170600, 717995, 357042, 173007, 819296, 252563, 807093,
        484226,   4861, 861254, 792930, 778791, 250004,  94299, 817511, 416668,
        787394, 819172,  10723, 460068, 170547, 522535, 560899,  67905,  92333,
         88598, 498621, 560310, 348197, 708012, 787196, 313623, 712585, 405548,
        733181, 594586, 228395, 595665, 255407, 878264, 572620, 811249, 127682,
          6460, 226774,  11796, 786224, 776052,  38506, 633519, 514078, 333475,
        496229, 158783, 350328, 450630,  29206, 608013,  87755, 393672,  66659,
        791344, 452087, 882379, 687447, 131698, 573455, 354961, 585695, 406704,
        569486, 578371, 807503,  87980, 233230, 533432, 729124, 107085, 417206,
         96554, 169045, 779354,

In [4]:
import implicit

ui_model = implicit.als.AlternatingLeastSquares(factors=100, use_gpu=False)
ai_model = implicit.als.AlternatingLeastSquares(factors=100, use_gpu=False)

ui_model.fit(ui_data)
ai_model.fit(ai_data)

NameError: name 'ui_data' is not defined

In [65]:
test_sim_rec_df = pd.read_csv(os.path.join(data_dir, "test_sim_recs.csv"), index_col=0)
test_sim_rec_df["sim_pids"] = test_sim_rec_df["sim_pids"].apply(lambda x: eval(x))
test_sim_rec_df = test_sim_rec_df.explode("sim_pids")
test_sim_rec_df["sim_pids"] = test_sim_rec_df["sim_pids"].astype("int64")
test_sim_rec_df = test_sim_rec_df.drop_duplicates(["uid", "aid", "sim_pids"])

test_ui_reldata = {}
test_ai_reldata = {}
for i, row in tqdm(test_sim_rec_df.iterrows(), total=len(test_sim_rec_df)):
    uid, aid, simpid = row.uid, row.aid, row.sim_pids
    if uid not in test_ui_reldata:
        test_ui_reldata[uid] = {}
        test_ui_reldata[uid][simpid] = 1.
    else:
        test_ui_reldata[uid][simpid] = 1.
    if aid not in test_ai_reldata:
        test_ai_reldata[aid] = {}
        test_ai_reldata[aid][simpid] = 1.
    else:
        test_ai_reldata[aid][simpid] = 1.
            
test_uids = np.array(test_sim_rec_df.uid.unique())
test_aids = np.array(test_sim_rec_df.aid.unique())

def get_ranking(model, uids, batch_size=2048, topk=1000):
    uid_to_ranklist = {}
    start_idx = 0
    while start_idx < len(uids):
        end_idx = min(len(uids), start_idx+batch_size)
        batch_uids = uids[start_idx: end_idx]
        score = -model.user_factors[batch_uids] @ model.item_factors.T # [bz, num_items]
        top_indices = np.argpartition(score, topk, axis=-1)
        batch_top_pids = top_indices[:, :topk]
        #top_pids = top_indices[score[top_indices].argsort()]
        
        for i, (uid, top_pids) in enumerate(zip(batch_uids, batch_top_pids)):
            assert len(top_pids) == topk
            uid_to_ranklist[uid] = top_pids[score[i][top_pids].argsort()]
        
        start_idx = end_idx
        print(start_idx, start_idx / len(uids))
            
    return uid_to_ranklist

uid_to_ranklist = get_ranking(ui_model, test_uids)
aid_to_ranklist = get_ranking(ai_model, test_aids)

100%|██████████| 109379/109379 [00:06<00:00, 16145.86it/s]


2048 0.025078369905956112
4096 0.050156739811912224
6144 0.07523510971786834
8192 0.10031347962382445
10240 0.12539184952978055
12288 0.15047021943573669
14336 0.1755485893416928
16384 0.2006269592476489
18432 0.22570532915360503
20480 0.2507836990595611
22528 0.27586206896551724
24576 0.30094043887147337
26624 0.32601880877742945
28672 0.3510971786833856
30720 0.3761755485893417
43008 0.5266457680250783
45056 0.5517241379310345
47104 0.5768025078369906
49152 0.6018808777429467
51200 0.6269592476489029
53248 0.6520376175548589
55296 0.677115987460815
57344 0.7021943573667712
59392 0.7272727272727273
61440 0.7523510971786834
63488 0.7774294670846394
65536 0.8025078369905956
67584 0.8275862068965517
69632 0.8526645768025078
71680 0.877742946708464
73728 0.9028213166144201
75776 0.9278996865203761
77824 0.9529780564263323
79872 0.9780564263322884
81664 1.0
2048 0.05029716587258706
4096 0.10059433174517413
6144 0.1508914976177612
8192 0.20118866349034825
10240 0.2514858293629353
12288 0.30

In [67]:
import sys 
def _calculate_metrics_plain(ranking, qrels,binarization_point=1.0,return_per_query=False):
    '''
    calculate main evaluation metrics for the given results (without looking at candidates),
    returns a dict of metrics
    '''

    ranked_queries = len(ranking)

    qidx_to_qid = {idx:qid for idx, qid in enumerate(ranking)}

    rr_per_candidate_depth = np.zeros((2,ranked_queries))
    rank_per_candidate_depth = np.zeros((2,ranked_queries))
    recall_per_candidate_depth = np.zeros((2,ranked_queries))
    ndcg_per_candidate_depth = np.zeros((2,ranked_queries))
    ap_per_candidate_depth = np.zeros((ranked_queries))
    evaluated_queries = 0

    for query_index,(query_id,ranked_doc_ids) in enumerate(ranking.items()):
        if query_id in qrels:
            evaluated_queries += 1

            relevant_ids = np.array(list(qrels[query_id].keys())) # key, value guaranteed in same order
            relevant_grades = np.array(list(qrels[query_id].values()))
            sorted_relevant_grades = np.sort(relevant_grades)[::-1]

            num_relevant = relevant_ids.shape[0]
            np_rank = np.array(ranked_doc_ids)
            relevant_mask = np.in1d(np_rank,relevant_ids) # shape: (ranking_depth,) - type: bool

            binary_relevant = relevant_ids[relevant_grades >= binarization_point]
            binary_num_relevant = binary_relevant.shape[0]
            binary_relevant_mask = np.in1d(np_rank,binary_relevant) # shape: (ranking_depth,) - type: bool

            # check if we have a relevant document at all in the results -> if not skip and leave 0 
            if np.any(binary_relevant_mask):

                # now select the relevant ranks across the fixed ranks
                ranks = np.arange(1,binary_relevant_mask.shape[0]+1)[binary_relevant_mask]

                #
                # ap
                #
                map_ranks = ranks[ranks <= 1000]
                ap = np.arange(1,map_ranks.shape[0]+1) / map_ranks
                ap = np.sum(ap) / binary_num_relevant
                ap_per_candidate_depth[query_index] = ap

                # mrr only the first relevant rank is used
                first_rank = ranks[0]

                for cut_indx, cutoff in enumerate([10, 1000]):

                    curr_ranks = ranks.copy()
                    curr_ranks[curr_ranks > cutoff] = 0 
                    #
                    # mrr
                    #

                    # ignore ranks that are out of the interest area (leave 0)
                    if first_rank <= cutoff: 
                        rr_per_candidate_depth[cut_indx,query_index] = 1 / first_rank
                        rank_per_candidate_depth[cut_indx,query_index] = first_rank

                for cut_idx, cutoff in enumerate([10,1000]):
                    curr_ranks = ranks.copy()
                    curr_ranks[curr_ranks > cutoff] = 0 
                    recall = (curr_ranks > 0).sum(axis=0) / binary_num_relevant
                    recall_per_candidate_depth[cut_idx,query_index] = recall

            if np.any(relevant_mask):

                # now select the relevant ranks across the fixed ranks
                ranks = np.arange(1,relevant_mask.shape[0]+1)[relevant_mask]

                grades_per_rank = np.ndarray(ranks.shape[0],dtype=int)
                for i,id in enumerate(np_rank[relevant_mask]):
                    grades_per_rank[i]=np.where(relevant_ids==id)[0]

                grades_per_rank = relevant_grades[grades_per_rank]

                #
                # ndcg = dcg / idcg 
                #
                for cut_indx, cutoff in enumerate([10,1000]):
                    #
                    # get idcg (from relevant_ids)
                    idcg = (sorted_relevant_grades[:cutoff] / np.log2(1 + np.arange(1,min(num_relevant,cutoff) + 1)))

                    curr_ranks = ranks.copy()
                    curr_ranks[curr_ranks > cutoff] = 0 

                    #coverage_per_candidate_depth[cut_indx, query_index] = (curr_ranks > 0).sum() / float(cutoff)

                    with np.errstate(divide='ignore', invalid='ignore'):
                        c = np.true_divide(grades_per_rank,np.log2(1 + curr_ranks))
                        c[c == np.inf] = 0
                        dcg = np.nan_to_num(c)

                    nDCG = dcg.sum(axis=-1) / idcg.sum()

                    ndcg_per_candidate_depth[cut_indx,query_index] = nDCG

    #avg_coverage = coverage_per_candidate_depth.sum(axis=-1) / evaluated_queries
    mrr = rr_per_candidate_depth.sum(axis=-1) / evaluated_queries
    relevant = (rr_per_candidate_depth > 0).sum(axis=-1)
    non_relevant = (rr_per_candidate_depth == 0).sum(axis=-1)

    """
    avg_rank=np.apply_along_axis(lambda v: np.mean(v[np.nonzero(v)]), -1, rank_per_candidate_depth)
    avg_rank[np.isnan(avg_rank)]=0.

    median_rank=np.apply_along_axis(lambda v: np.median(v[np.nonzero(v)]), -1, rank_per_candidate_depth)
    median_rank[np.isnan(median_rank)]=0.
    """
    map_score = ap_per_candidate_depth.sum(axis=-1) / evaluated_queries
    recall = recall_per_candidate_depth.sum(axis=-1) / evaluated_queries
    nDCG = ndcg_per_candidate_depth.sum(axis=-1) / evaluated_queries

    local_dict={}

    for cut_indx, cutoff in enumerate([10,1000]):

        local_dict['MRR@'+str(cutoff)] = mrr[cut_indx]
        #local_dict['QueriesWithNoRelevant@'+str(cutoff)] = non_relevant[cut_indx]
        local_dict['QueriesWithRelevant@'+str(cutoff)] = relevant[cut_indx]
        #local_dict['AverageRankGoldLabel@'+str(cutoff)] = avg_rank[cut_indx]
        #local_dict['MedianRankGoldLabel@'+str(cutoff)] = median_rank[cut_indx]

    for cut_indx, cutoff in enumerate([10,1000]):
        local_dict['Recall@'+str(cutoff)] = recall[cut_indx]

    for cut_indx, cutoff in enumerate([10,1000]):
        #local_dict['Avg_coverage@'+str(cutoff)] = avg_coverage[cut_indx]
        local_dict['nDCG@'+str(cutoff)] = nDCG[cut_indx]

    local_dict["MAP@"+str(1000)] = map_score

    local_dict['QueriesRanked'] = evaluated_queries

    if return_per_query:
        return local_dict,rr_per_candidate_depth,recall_per_candidate_depth,ndcg_per_candidate_depth, qidx_to_qid, qrels
    else:
        return local_dict
        
ui_result = _calculate_metrics_plain(uid_to_ranklist, test_ui_reldata)
ai_result = _calculate_metrics_plain(aid_to_ranklist, test_ai_reldata)

In [69]:
ui_result, ai_result

({'MRR@10': 0.12848136732161516,
  'QueriesWithRelevant@10': 19666,
  'MRR@1000': 0.13421565944717023,
  'QueriesWithRelevant@1000': 43670,
  'Recall@10': 0.2148971226235994,
  'Recall@1000': 0.5136589299708912,
  'nDCG@10': 0.14111959727735726,
  'nDCG@1000': 0.19248874069304256,
  'MAP@1000': 0.11840409862342591,
  'QueriesRanked': 81664},
 {'MRR@10': 0.1465949792494564,
  'QueriesWithRelevant@10': 9988,
  'MRR@1000': 0.15065814610725997,
  'QueriesWithRelevant@1000': 15242,
  'Recall@10': 0.170981202532537,
  'Recall@1000': 0.35062336823934886,
  'nDCG@10': 0.12993301089907136,
  'nDCG@1000': 0.17442168253031834,
  'MAP@1000': 0.10880731329619221,
  'QueriesRanked': 40718})